In [ ]:
# 自然言語でデータベースを検索する
# 参考：LangChainを使って自然言語でデータベースを検索する（Azure編）（https://zenn.dev/cannoteat/articles/ae6040bfaa52d4）

# 方法１（リレーショナルデータベース）
# chatgptで入力からsql文を生成し、sql文を実行し、そこからランダムに返す

# 方法2 （ベクトルデータベース）
# ベクトルデータで入力をベクトル化し、ベクトルデータベースの中から類似度が高いものを返す
# 参考：OpenAIのEmbeddings APIのベクトルを使って検索を行う（https://dev.classmethod.jp/articles/search-with-openai-embeddings/）

In [15]:
# 方法0 簡易な方法
import openai
import os, json
from pprint import pprint

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

user_input = "奈良で有名な絵画が展示される展示会"


system = "あなたは博識で優秀な博物館のキュレーターです。"
exhibition_input = """以下は開催中の展示物の一覧です。以下の質問についてこの表を使用して答えてください。
展示番号    館名	都道府県	展示名	会期(始)	会期(終)	公式サイトURL	説明文	分類番号
01  四日市市立博物館	三重県	開館30周年記念　特別展 立原位貫　〜想像力から創造力へ〜	2023/9/23	2023/11/5	https://www.city.yokkaichi.mie.jp/museum/info/pg329.html	四日市ゆかりの木版画家・立原位貫（1951-2015）は、独学で江戸期浮世絵の魅力を現代に蘇らせ、さらにはオリジナルでも粋で個性的な作品を創り出しました。大英博物館に収蔵されるなど、世界に誇れる画家の足跡を辿り、その面白さ、美しさを紹介します。	73,09,721
03  パラミタ  ミュージアム	三重県	つながる琳派スピリット　神坂雪佳展	2023/10/6	2023/11/26	https://paramitamuseum.com/plan/next_exhibition.html#plan01	近代京都において、琳派を手本として創作活動を行った神坂雪佳(1866～1942)。京に生まれた雪佳は、はじめ四条派の画法を学びますが、新しい時代にふさわしい工芸図案の必要性を認識し、図案家の岸光景に師事します。押し寄せる西洋文化を意識しながらも、雪佳は装飾芸術である「琳派」に傾倒し、図案家、画家として活躍しました。雪佳が手掛けたデザインは、染織、陶芸、漆芸から室内装飾や庭園まで実に多面的です。これは、雪佳が敬愛した本阿弥光悦や尾形光琳の柔軟な活動に倣うものといえるでしょう。琳派は、江戸初期の本阿弥光悦、俵屋宗達(生没年不詳)にはじまり、江戸中期の光琳やその弟・尾形乾山、江戸後期の酒井抱一らによって私淑という形で受け継がれてきました。平安以来の美意識をそなえた装飾性豊かな作風で、絵画や工芸といった領域を越え、さまざまな意匠を生み出してきた琳派は、近代以降も多くの分野にわたって影響を与え続けています。本展では、雪佳が手本とした琳派の美の潮流を本阿弥光悦や尾形光琳らの名品にたどるとともに、古典と近代的発想を融合させ、美術と意匠の二つの分野を自在に往来した近代琳派・神坂雪佳の多彩な世界をご覧いただきます。	21,12,70,721,727,750,757
04  パラミタ  ミュージアム	三重県	Sasa Adairコレクション 20世紀巨匠の版画達展 シャガール、ピカソ、ダリからロックウェルまで	2023/12/1	2024/1/29	https://paramitamuseum.com/plan/schedule.html	マティス、ブラックからビュッフェの時代まで、彼らが信頼を寄せたパリの版画工房や出版社の協力で制作されたリトグラフ。アメリカンイラストレーションの黄金時代を築いたパリッシュ、古き良きアメリカを象徴するロックウェルによる雑誌の表紙絵。それらの版画約130点を展観いたします。ヨーロッパとアメリカの“版画達”を集めて帰国した「Sasa Adairコレクション」を公開。誰もが聞いたことがある作家、教科書で見たことがある作品をみつけながら、20世紀の芸術をお楽しみください。	25,23,73,74,727
06  桑名市博物館	三重県	特別企画展 三方領知替200年　行田市・桑名市・白河市友好都市締結25周年記念合同企画展「武門の遺産ー徳川家を支えた忍・桑名・白河ー」	2023/10/28	2023/11/26	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2023calendar.html	文政6年（1823）に忍藩主阿部正権が白河へ、白河藩主松平定永が桑名へ、桑名藩主松平忠堯が忍へ国替えとなる三方領知替（さんぽうりょうちがえ）が行われました。この国替えが縁となり、平成10年（1998）に行田市・桑名市・白河市の三市は友好都市を締結しました。令和5年（2023）は三方領知替200年、友好都市締結25周年という節目の年となります。三市の歴史・文化的魅力を広く市民や各市を訪れる人々に伝える絶好の機会であることから、三市の所蔵する「おたから」を一堂に会する展覧会を実施し、三市の文化財の魅力を各市において紹介し、今後の観光交流を推進していく上での一助とします。	21,09,721,728,750
07  桑名市博物館	三重県	企画展「BE BRAVE ―ヴィアティン三重、2023年の軌跡―」	2023/12/9	2023/12/17	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2024calendar.html	2023年のシーズンを振り返る企画展を開催いたします。	78
09  桑名市博物館	三重県	春季企画展「刀剣アラカルト」	2024/3/9	2024/5/12	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2026calendar.html	館蔵の刀剣を中心に、一振一振異なる味わいをご紹介します。一つとして同じもののない刀剣の魅力をご堪能ください。	21,756
10  公益財団法人松伯美術館	奈良県	文化勲章受章・卒寿記念特別展 上村淳之展	2023/10/14	2023/12/24	https://www.kintetsu-g-hd.co.jp/culture/shohaku/exhibition/exhibition.html	令和５年、当館の館長・上村淳之は卒寿を迎えます。淳之は、幼いころから、父 松篁が飼っていた小鳥たちとふれあい、美大三年生までの課題制作をおえると、専ら花鳥画を描くようになります。「鳥が好きだからテーマになり、テーマになるから鳥の数も増えてゆく」と述べるように、「唳禽荘」（自宅・アトリエ）で多くの鳥と共に暮らし、絵筆を握ってきました。その業績により、平成25年（2013）に文化功労者となり、令和４年（2022）文化勲章を受章しました。 東洋的感性の減退や現代の日本画について真摯に警告を発し、さまざまな鳥の姿を通じて、自然の叡智を示し、未来へのメッセージを送り続ける上村淳之の姿を紹介します。	
17  公益財団法人大和文華館	奈良県	文字を愛でる　―経典・文学・手紙から―	2024/2/23	2024/4/7	https://www.kintetsu-g-hd.co.jp/culture/yamato/exhibition/	中国や日本の書の文化は、書体や書風を多彩に展開し、文字を芸術の域にまで昇華させました。当館が所蔵する経典や文学作品、手紙を通して、美しい文字の世界へご案内します。	

"""

response_format = """回答は以下の形式で行ってください。
[{
    "exhibition_number": {展示番号},
    "exhibition_name": {展示名},
    "exhibition_reason": {その展示を選んだ理由}
},
~~~
~~~
~~~
{
    "exhibition_number": {展示番号},
    "exhibition_name": {展示名},
    "exhibition_reason": {その展示を選んだ理由}
}]
"""


question = "以下に与えられる入力をもとにおすすめの展示を選んでください。\n" + user_input

messages = [
    {"role": "system", "content": system},
    {"role": "system", "content": exhibition_input},
    {"role": "system", "content": response_format},
    {"role": "user", "content": question}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)
result = response.choices[0]["message"]["content"].strip()
result_json = json.loads(result)
pprint(result_json)


[{'exhibition_name': '文化勲章受章・卒寿記念特別展 上村淳之展',
  'exhibition_number': 10,
  'exhibition_reason': '上村淳之は日本の花鳥画家として有名であり、その作品は美しく優雅な絵画で知られています。また、文化勲章を受章した著名な芸術家です。奈良県の公益財団法人松伯美術館で開催されるこの特別展では、上村淳之の作品をじっくりと鑑賞することができます。奈良で有名な絵画を楽しみたい方におすすめの展示です。'},
 {'exhibition_name': '文字を愛でる\u3000―経典・文学・手紙から―',
  'exhibition_number': 17,
  'exhibition_reason': '奈良県の公益財団法人大和文華館で開催されるこの展示は、中国や日本の書の文化をテーマにしています。美しい文字の世界を味わうことができる経典や文学作品、手紙が展示されています。奈良で有名な絵画だけでなく、文字の美しさにも興味がある方におすすめです。'}]


In [6]:
# 方法2：ベクトルデータベースの検索
import openai
from openai.embeddings_utils import cosine_similarity
import os,json

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

# データベースの読み込み
with open('embedding.json') as f:
    INDEX = json.load(f)

# これが検索用の文字列
QUERY = '関西で有名な絵画が展示される展示会'

# 検索用の文字列をベクトル化
query = openai.Embedding.create(
    model='text-embedding-ada-002',
    input=QUERY
)

query = query['data'][0]['embedding']

# 総当りで類似度を計算
results = map(
    lambda i: {
        'number': i['number'],
        'name': i['name'],
        'raw': i['raw'],
        'embedding_text': i['embedding_text'],
        # ここでクエリと各文章のコサイン類似度を計算
        'similarity': cosine_similarity(i['embedding'], query)
        },
    INDEX
)
# コサイン類似度で降順（大きい順）にソート
results = sorted(results, key=lambda i: i['similarity'], reverse=True)

# 以下で結果を表示
print(f"Query: {QUERY}")
print("Rank: Title Similarity")
for i, result in enumerate(results):
    print(f'{i+1}: {result["number"]} {result["raw"][1]} {result["name"]} {result["similarity"]}')

print("====Best Doc====")
print(f'number: {results[0]["number"]}')
print(f'name: {results[0]["name"]}')

print("====Worst Doc====")
print(f'number: {results[-1]["number"]}')
print(f'name: {results[-1]["name"]}')

Query: 関西で有名な絵画が展示される展示会
Rank: Title Similarity
1: 73 奈良県 近代の日本画〈日本画展示室〉 0.9113036395213371
2: 84 和歌山県 小企画展「原勝四郎と同時代の画家たち」 0.8947766196284109
3: 76 奈良県 特別企画展　やまと絵のこころ 0.894100672799945
4: 72 奈良県 松本竣介、駒井哲郎から舟越桂まで－戦後日本の洋画・版画－〈洋画展示室〉 0.8929414852733095
5: 71 奈良県 企画展　未来につなぐ日本画展 －松伯美術館公募展優秀賞受賞作家の現在Ⅱ－ 0.8881885814638384
6: 31 京都府 白沙村荘秋季展　2023　「名品よ、再び。」 0.8875822303635622
7: 1 三重県 開館30周年記念　特別展 立原位貫　〜想像力から創造力へ〜 0.8874311468997833
8: 36 京都府 浜村満果 個展「9 Hours Sleep」 0.8868082630404737
9: 83 和歌山県 原勝四郎展 　南海の光を描く 0.8864869533262518
10: 33 京都府 表装の愉しみ －ある表具師のものがたり 0.8817645514549303
11: 38 大阪府 安野光雅展 0.8810993240232374
12: 65 奈良県 特別展　漂泊の画家　不染鉄 0.8798356142783563
13: 5 三重県 隙あらば猫 町田尚子絵本原画展 0.8795057094278214
14: 3 三重県 つながる琳派スピリット　神坂雪佳展 0.8780155559908251
15: 67 奈良県 おん祭と春日信仰の美術 0.8775901032376583
16: 54 兵庫県 司馬遼太郎生誕百年　企画展「小林修写真展　司馬遼太郎『街道をゆく』の視点　－歩いた風土、見抜いた時代－」 0.8765350786219296
17: 57 兵庫県 第59回 姫路市児童生徒科学作品展　調査研究の部 0.8762715903493459
18: 9 三重県 春季企画展「刀剣アラカルト」 0.8754749442874001
19: 35 京都府 京都精華大学55周年記念展「FATHOM